In [1]:
import pandas as pd
import os
import math
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, Ridge
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from textblob import TextBlob
# Use a pipeline as a high-level helper
from transformers import pipeline

In [2]:
all_movies_df = pd.read_csv('all_movies.csv')
all_movies_df = all_movies_df.dropna(subset=['MovieBoxOffice', 'MoviePlot']).reset_index(drop=True)
all_movies_df

,MovieName,MovieYear,MovieBoxOffice,MovieRuntime,MovieLanguage,MovieCountries,MovieGenre,MovieRating,MovieRatingNb,MoviePlot
0,The Corbett-Fitzsimmons Fight,1897,100000.0,NaN,NaN,NaN,Sports,5.2,541,The film no longer exists in its entirety; how...
1,The Million Dollar Mystery,1914,1500000.0,NaN,English Language,United States of America,"Silent film, Indie, Black-and-white, Adventure",5.9,43,"The serial follows a secret society called ""Th..."
2,Rose of the Rancho,1914,87028.0,NaN,"Silent film, English Language",United States of America,"Western, Silent film, Black-and-white, Indie",3.2,48,"Esra Kincaid takes land by force and, having t..."
3,The Birth of a Nation,1915,50000000.0,190.0,"Silent film, English Language",United States of America,"Silent film, Indie, Costume drama, Epic, Black...",6.1,26959,The film follows two juxtaposed families: the...
4,Way Down East,1920,4500000.0,147.0,"Silent film, English Language",United States of America,"Silent film, Indie, Black-and-white, Action/Ad...",7.3,6027,"The rich, typified by the handsome man-about-t..."
...,...,...,...,...,...,...,...,...,...,...
6904,Over My Dead Body,2012,6373398.0,110.0,Korean Language,South Korea,"Crime Fiction, Drama",6.1,365,Baek Hyun-chul is a biotech scientist researc...
6905,The Future,2011,568290.0,91.0,English Language,"United States of America, Germany","Comedy film, Drama, Indie",5.3,19,The cinematic translation involves a couple in...
6906,Quarantine,2008,41319906.0,89.0,English Language,United States of America,"Thriller, Science Fiction, Horror, Zombie Film...",3.9,201,{{plot}} A television reporter named Angela Vi...
6907,The Return of Godzilla,1984,4116395.0,103.0,"Japanese Language, Russian Language, English L...",Japan,"Science Fiction, Japanese Movies, Monster",6.8,2830,A Japanese fishing vessel is trying to find it...


In [3]:
adapted_movies_df = pd.read_csv('adapted_movies.csv')
adapted_movies_df = adapted_movies_df.dropna(subset=['MovieBoxOffice', 'MoviePlot']).reset_index(drop=True)
adapted_movies_df

,MovieName,MovieYear,MovieBoxOffice,MovieRuntime,MovieLanguage,MovieCountries,MovieGenre,BookAuthor,BookTitle,BookStartYear,BookEndYear,BookDescription,BookRating,BookRatingNb,BookLanguage,MovieRating,MovieRatingNb,MoviePlot
0,The Birth of a Nation,1915,50000000.0,190.0,"Silent film, English Language",United States of America,"Silent film, Indie, Costume drama, Epic, Black...","Thomas Dixon, Jr.",The Clansman: A Historical Romance of the Ku K...,1905,1905,NaN,NaN,NaN,NaN,6.1,26959,The film follows two juxtaposed families: the...
1,Little Lord Fauntleroy,1921,900000.0,112.0,"Silent film, English Language",United States of America,"Silent film, Drama, Comedy, Black-and-white, F...",Frances Hodgson Burnett,Little Lord Fauntleroy,1885-1886,1885-1886,Illustrated. Frances Hodgson Burnett's charact...,3.85,18137.0,NaN,6.8,384,Cedric Errol is a poor American boy who finds ...
2,Greed,1924,274827.0,246.0,"Silent film, English Language",United States of America,"Silent film, Drama, Black-and-white",Frank Norris,McTeague,1899,1899,NaN,3.68,5410.0,eng,8.0,11078,{{quote box}} Trina & McTeague The film opens ...
3,The Phantom of the Opera,1925,2000000.0,106.0,"Silent film, English Language, American English","United States of America, England","Horror, Silent film, Drama",Gaston Leroux,The Phantom of the Opera,1909-1910,1909-1910,NaN,3.97,189814.0,NaN,7.5,20708,:The scenario presented is based on the genera...
4,Beau Ideal,1931,575000.0,NaN,NaN,United States of America,"Romance Film, Adventure, War film",P. C. Wren,Beau Geste,1924,1924,Three noble English brothers battle a sadistic...,4.01,3047.0,NaN,4.9,251,"The hero, John Geste, joins the French Foreign..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,White Night,2009,6171057.0,135.0,Korean Language,South Korea,"Thriller, Mystery, Romance Film",Keigo Higashino,Journey Under the Midnight Sun,1999,1999,NaN,NaN,NaN,NaN,6.5,1475,"Kim Si-hoo, a pawnbroker is found murdered in ..."
998,Into The White Night,2010,4593914.0,149.0,Japanese Language,Japan,"Drama, Japanese Movies, World cinema",Keigo Higashino,Journey Under the Midnight Sun,1999,1999,NaN,NaN,NaN,NaN,6.5,518,"A pawn shop owner in Osaka is murdered, but du..."
999,The Client,2011,15780280.0,122.0,Korean Language,South Korea,"Crime Fiction, Mystery, Thriller, Drama",John Grisham,The Client,1993,1993,NaN,4.01,378641.0,eng,6.8,2040,"On his wedding anniversary, Han Chul-min driv..."
1000,Helpless,2012,16175929.0,117.0,Korean Language,South Korea,"Thriller, Mystery",Miyuki Miyabe,All She Was Worth,1992,1992,Here is a deftly written thriller that is also...,3.69,2921.0,NaN,6.7,2662,"South Korea, 2009. A few days before their wed..."


### sentiment extraction

### output : (take technique with best R^2 value or all)
- plot data visu with param : feature to plot
- plot coefs bar with intervals with param : genre (or overall)
- overall comparison between overall between all_movies and adapted_movies of coefs with param: technique
- correlation coefficient between book plot and movie plot (pearson and non-linear one)


In [4]:
def sentiment_analysis(pipeline, df):  
    box_office = df.MovieBoxOffice
    sentiments = pd.DataFrame(pipeline(df.MoviePlot).tolist())
    return pd.concat([box_office, sentiments], axis=1)


##### vader

In [ ]:
def vader_sentiment_analysis(texts):
    analyzer = SentimentIntensityAnalyzer()
    sentiments = texts.apply(lambda t: analyzer.polarity_scores(t))
    return sentiments

##### transformers

In [41]:
def BERT_sentiment_analysis(texts):
    classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
    predictions = classifier(texts.tolist())
    sentiments = pd.DataFrame([{sent['label']: sent['score'] for sent in pred} for pred in predictions])
    return sentiments

##### textblob

In [8]:
def textblob_sentiment_analysis(texts):
    sentiments = texts.apply(lambda t: {"polarity" : TextBlob(t).sentiment.polarity,
                                        "subjectivity" : TextBlob(t).sentiment.subjectivity})
    return sentiments

### linear regression

In [ ]:
vader_sentiments = sentiment_analysis(vader_sentiment_analysis, adapted_movies_df)
BERT_sentiments = sentiment_analysis(BERT_sentiment_analysis, adapted_movies_df)
textblob_sentiments = sentiment_analysis(textblob_sentiment_analysis, adapted_movies_df)

In [43]:
# plot box-office to sentiment
def exploratory_plot(data, y):
    for column in data.columns:
        if column == y:
            continue
        data.plot(kind='scatter', x=column, y=y, figsize=(16, 5), grid=True)
        plt.show()
    return

In [ ]:
exploratory_plot(data, 'MovieBoxOffice')

In [ ]:
# NOTE: find way to do feature augmentation better ! 

data['compound_aug'] = data.compound.apply(lambda x: x**2)

In [ ]:
def train_linear_regression(data):

    target = 'MovieBoxOffice'
    features = data.drop(columns = [target]).columns.tolist() 
    formula = f"{target} ~ {' + '.join(features)}"

    model = smf.ols(formula=formula, data=data)
    np.random.seed(2)
    results = model.fit()
    print(results.summary())

    return results


In [ ]:
data = sentiment_analysis(vader_sentiment_analysis, adapted_movies_df)
results = train_linear_regression(data)
# plot coefs of each sentiment for a genre
print(results.summary())


In [31]:
data = sentiment_analysis(vader_sentiment_analysis, adapted_movies_df)
data

,MovieBoxOffice,neg,neu,pos,compound
0,50000000.0,0.091,0.813,0.096,0.9099
1,900000.0,0.108,0.804,0.087,-0.1531
2,274827.0,0.140,0.800,0.060,-0.9992
3,2000000.0,0.102,0.796,0.102,0.5619
4,575000.0,0.062,0.770,0.168,0.9557
...,...,...,...,...,...
997,6171057.0,0.175,0.779,0.046,-0.9870
998,4593914.0,0.180,0.784,0.036,-0.9501
999,15780280.0,0.144,0.805,0.051,-0.9764
1000,16175929.0,0.107,0.872,0.021,-0.9717
